In [ ]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
import pandas as pd
import regex as re
import numpy as np
import requests
import string
import time 

domain = 'http://amazon.com'
URL = 'https://www.amazon.com/s?i=beauty&bbn=11060451&rh=n%3A3760911%2Cn%3A11055981%2Cn%3A11060451%2Cp_n_target_audience_browse-bin%3A405780011&dc&page=148&fst=as%3Aoff&qid=1599987405&rnid=405778011&ref=sr_pg_147'
driver = webdriver.Chrome(executable_path ='/Applications/chromedriver')
driver.get(URL)
subhtml = driver.page_source
soup = BeautifulSoup(subhtml, "html.parser")

while True:
    
    for i in soup.find_all(class_='a-section a-spacing-medium'):
        
        #go inside product page
     
        nextlink = domain + i.find(class_='a-link-normal a-text-normal')['href']
        print(nextlink)
        driver.get(nextlink)
        temp = driver.page_source
        soup = BeautifulSoup(temp, "html.parser")

        #1. extract brand name
        try:
            brand = soup.find_all(id='bylineInfo')[0].text
            if 'Visit' in brand:
                brand = re.match('(?:.{0,10})(.+)(?:\s.+)', brand).group(1)
                print(brand)
            elif 'Brand:' in brand:
                brand = re.match('^Brand\:\s(.+)', brand).group()
            else:
                print('NO brand name')
        except:
            brand = np.nan
            print('NO Brand')
            
        
        #2. extract product name        
        try:   
            product_name = soup.find(id='productTitle').text.replace('\n','').replace(brand+' ','')
            print(product_name)
        except:
            product_name = np.nan
            print('NO product_name')
            
        #3. extract category
        try:
            cate = soup.find(id='wayfinding-breadcrumbs_feature_div').find_all(class_='a-list-item')[6].text.replace('\n','')
            print('Categorie: '+cate)
        except:
            cate = np.nan
            print('NO CATE') 

            
        #4. extract original price /rmb covert to hkd!
        try:
            original_price = soup.find(class_='priceBlockStrikePriceString a-text-strike').text
            print('original price(USD): '+original_price)
            
        except:
            
            try:
                original_price = soup.find(class_='a-size-medium a-color-price priceBlockBuyingPriceString').text
                print('original price(USD): '+original_price)
            except:
                original_price = np.nan
                print('NO original_price')
                
               
                
        #5. extract discounted price /rmb covert to hkd!
        try:
            discounted_price = soup.find(class_='a-size-medium a-color-price priceBlockBuyingPriceString').text
            print('discounted_price(USD): '+discounted_price)
            
        except:
            
            try:
                discounted_price = original_price
                print('discounted_price(USD): '+discounted_price)
            except:
                discounted_price = np.nan
                print('NO discounted_price')
                
        #6. extract promotiona message
        try:
            promotion_message = soup.find_all(id='promoPriceBlockMessage_feature_div')[1].text.replace('\n','').replace('  Shop items','')
            print('Promotion Message: '+promotion_message)
        except:
            try:
                promotion_message = soup.find(id='vpcButton').text.replace('\n','')
                print('Promotion Message: '+promotion_message)
            except:
                promotion_message = np.nan
                print('NO Promo')
 
        #7. extract no of reviews(ratings)
        try:
            no_of_reviews = soup.find(id='acrCustomerReviewText').text.replace(',',"")
            no_of_reviews = re.match('(\d+)', no_of_reviews).group()
            print('no_of_reviews: '+no_of_reviews)
        except:
            no_of_reviews = np.nan
            print('NO no_of_reviews')


        #8. extract ratings    
        try:
            ratings = soup.find(class_='a-icon-alt').text
            ratings = re.match('(\d\.?\d?)(?:\s.+)', ratings).group(1)
            print('ratings: ' + ratings)
        except:
            ratings = np.nan
            print('NO ratings')
        
        df = df.append({'Brand':brand,
                        'Product Name':product_name,
                        'Category':cate,
                        'Original Price':original_price,
                        'Discounted Price':discounted_price,
                        'No. of Reviews':no_of_reviews,
                        'Ratings':ratings,
                        'Promotion Message':promotion_message}, ignore_index = True)

        
    #going back to main page after looping    
    temp = driver.get(URL) 
    subhtml = driver.page_source
    soup = BeautifulSoup(subhtml, "html.parser")

    #going to the next page
    temp2 = soup.find(class_='a-last').a['href']
    URL = domain + temp2
    driver.get(URL)
    subhtml = driver.page_source
    soup = BeautifulSoup(subhtml, "html.parser")

# df.to_csv("amazon.csv")